## Training process for ECD507 Senior Capstone Project - F1Tenth ML Based Autonomous Race Car
### Contributors - Charles Hodgins, Rishabh Hegde, Dylan DiGiacomo, and Andrew Meccariello

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import ast

In [2]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.6


In [11]:
class DrivingDataset(Dataset):
    def __init__(self, csv_file):
        self.csv = pd.read_csv(csv_file, chunksize=10000)
        self.data = next(self.csv);
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx,1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (200, 66))  # Resize using OpenCV
        image = image / 255.0  # Normalize pixel values to [0,1]
        image = np.transpose(image, (2, 0, 1))  # Change shape to (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        steering_angle = float(self.data.iloc[idx, 3])
        throttle = float(self.data.iloc[idx, 4])
        lidar = torch.tensor(ast.literal_eval(self.data.iloc[idx, 2]))
        imu_rotation = float(self.data.iloc[idx,5])
        imu_linear = float(self.data.iloc[idx,6])
               
        return image, lidar, torch.tensor([steering_angle, throttle],dtype=torch.float32),torch.tensor([imu_rotation,imu_linear], dtype=torch.float32)


In [4]:
df = pd.read_csv('/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log.csv')
print(len(df))

103213


In [10]:
def filter_chunk(df):
    global first_chunk
    print(first_chunk)
    print(f"Initial size: {len(df)}")

    def is_valid_image(img_path):
        return os.path.exists(img_path) and cv2.imread(img_path) is not None
    
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    before_throttle = len(df)
    df = df[df["throttle"] != 0 ]
    print(f"Filtered throttle: {before_throttle - len(df)} rows removed")

    before_lidar = len(df)
    df = df[df["lidar_list"] != ""]
    df = df[pd.notna(df["lidar_list"])]
    print(f"Filtered lidar: {before_lidar - len(df)} rows removed")

    before_images = len(df)
    df["image_path"] = df["image_path"].str.strip()
    df = df[df["image_path"] != ""]
    df_filtered = df[df["image_path"].apply(is_valid_image)]
    print(f"Filtered images: {before_images - len(df_filtered)} rows removed")

    print(f"Final size: {len(df_filtered)}")
    
    print(len(df_filtered))
    
    if first_chunk:
        df_filtered.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv", mode='w', index=False, header=True)
        first_chunk = False
    else:
        df_filtered.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv", mode='a', index=False, header=False)
    # Save the cleaned dataset
first_chunk = True

for chunk in pd.read_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log.csv",chunksize = 10000):
    print('loaded chunk')
    filter_chunk(chunk)
    # print(chunk.head())
    del chunk
    
print('done')

loaded chunk
True
Initial size: 10000
Filtered throttle: 1009 rows removed
Filtered lidar: 12 rows removed
Filtered images: 0 rows removed
Final size: 8979
8979
loaded chunk
False
Initial size: 10000
Filtered throttle: 1331 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 8669
8669
loaded chunk
False
Initial size: 10000
Filtered throttle: 371 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 9629
9629
loaded chunk
False
Initial size: 10000
Filtered throttle: 773 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 9227
9227
loaded chunk
False
Initial size: 10000
Filtered throttle: 2644 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 7356
7356
loaded chunk
False
Initial size: 10000
Filtered throttle: 1082 rows removed
Filtered lidar: 0 rows removed
Filtered images: 0 rows removed
Final size: 8918
8918
loaded chunk
False
Initial size: 100

KeyboardInterrupt: 

In [12]:
dataset = DrivingDataset('/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv')

dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [13]:
import torch
import torch.nn as nn

class IMUMLP(nn.Module):
    def __init__(self):
        super(IMUMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 64),  # 2 input features → 64 neurons
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 2 outputs (steering, throttle)
        )

    def forward(self, x):
        return self.model(x)
    
class CameraLidarFusion(nn.Module):
    def __init__(self):
        super(CameraLidarFusion, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 64),  # 2 input features → 64 neurons
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 2 outputs (steering, throttle)
        )

    def forward(self, x):
        return self.model(x)
    
class FullFusionModel(nn.Module):
    def __init__(self):
        super(CameraLidarFusion, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(6, 64),  # 6 predictions features → 64 neurons
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 2 outputs (steering, throttle)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:

## train for the fusion model of lidar and camera predictions
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change to cuda when it begins to work
print(len(dataset))
imu_model = IMUMLP().to(device)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(imu_model.parameters(), lr=0.001, )

# Training loop
epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for images,lidar,targets, imu in dataloader:
        
        imu, targets = imu.to(device), targets.to(device)        
        
        outputs = imu_model(imu)
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}')

# Save model
torch.save(imu_model.state_dict(), os.path.expanduser('/media/ecd507/JetsonOrinNano/home/ecd507/training/imu_model.pth'))
print("Model training complete and saved as imu_model.pth")

10000


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0xffffb2fe45b0>>
Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
